In [ ]:
import pandas as pd
import statistics
from z3 import *
from sklearn.preprocessing import MinMaxScaler
import random
import numpy as np


folders = range(1, 11)
dataset = 'music_data.csv' # not provided
data = pd.read_csv(dataset, header=0).iloc[:, 1:]

instances_to_change = [i for i in range(data.shape[0]) if data['group'][i] == 'The Beatles']
print(len(instances_to_change))
instances_to_change = random.sample(instances_to_change, 60)

for i in instances_to_change:
    data['Music Style'][i] = random.choice(['Metal', 'Electronic', 'Classical'])

X_original = data.iloc[:, :-1]
y = list(data.iloc[:, -1])

categorical_features = [f for f in list(X_original.columns) if f not in set(X_original._get_numeric_data().columns)]
numerical_features = list(set(X_original._get_numeric_data().columns))
scaler = MinMaxScaler()

X_original[numerical_features] = scaler.fit_transform(X_original[numerical_features])
X = pd.get_dummies(X_original, columns=categorical_features, drop_first=False)
print(X.head())

In [ ]:
# to count the number of violations in the data

def count_violations(X, y):
    count = 0
    for i in range(X.shape[0]):
        if X['group_The Beatles'][i] == 1:
            if y[i] in ['Electronic', 'Metal', 'Classical']:
                count += 1
    return count

count_violations(X, y)

In [ ]:
# to calculate average pairwise distance

from sklearn.metrics import pairwise_distances

def infy_pairwise_distance(a, b):
    return max(abs(a - b))

pairwise_distances(X, X, metric=infy_pairwise_distance).mean()

In [ ]:
# adversity index

def sumproduct(X, W):
    out = W[0]
    for i in range(len(X)):
        out = out + X[i] * W[i + 1]
    return out

def adversity_index(X, parameters=None, K=1, num=10):
    
    output = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
    if parameters is None:
        print('please provide a valid set of parameters')
        sys.exit()

    errors = [0.1, 0.01, 0.001]
    
    for n in range(len(errors)):
        
        print('running for error:', errors[n])
        s = Solver()
        _X = RealVector('x', X.shape[1])

        for i in range(X.shape[0]):
            
            if i % 500 == 0: print('tested for {} instances'.format(i))
            
            for k in [0, 1, 2, 3, 4, 5]:
                s.add(And(_X[k] <= max(list(X.iloc[:, k])), _X[k] >= min(list(X.iloc[:, k]))))

            for k in range(6, 12):
                s.add(_X[k] == 0)
            
            s.add([And(_X[j] > list(X.iloc[i, :])[j] - errors[n],
                       _X[j] < list(X.iloc[i, :])[j] + errors[n]) for j in range(X.shape[1])])

            if K is not None:
                s.add(And(_X[12] == 1, Or([sumproduct(_X, parameters[k]) > 0 for k in range(3)])))

            out = s.check()
            if out == sat:
                solution = [s.model()[x].numerator_as_long() / s.model()[x].denominator_as_long() for x in _X]
                output[n] = output[n].append([[i+1] + solution])
            s.reset()

        if output[n].shape[0] > 0:
            output[n].columns = ['original_data_instance'] + list(X.columns)
    return output            

In [ ]:
out_sbr = pd.DataFrame()
for f in folders:
    print('Running for Experiment:', f)
    baseline_results = pd.read_csv(str(f) + '/all_baseline_outputs.csv').iloc[:, 1:]
    baseline_results = baseline_results[(baseline_results['margin/alpha'] != 'PP') & (baseline_results['loss_type'] == 'sbr')]
    
    alphas = [0, 1, 2, 5, 10, 50, 100]
    processed_baseline_results = pd.DataFrame()

    for a in alphas:
        baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(a)]
        mean_accuracy_test = statistics.mean(list(baseline_results_sub['accuracy_test_corrected']))
        std_accuracy_test = statistics.stdev(list(baseline_results_sub['accuracy_test_corrected'])) 
        mean_accuracy_test_modified = statistics.mean(list(baseline_results_sub['accuracy_test_modified']))
        std_accuracy_test_modified = statistics.stdev(list(baseline_results_sub['accuracy_test_modified'])) 
        mean_val_violations = statistics.mean(list(baseline_results_sub['violations_val_prediction']))
        std_val_violations = statistics.stdev(list(baseline_results_sub['violations_val_prediction']))
        mean_test_violations = statistics.mean(list(baseline_results_sub['violations_test_prediction']))
        std_test_violations = statistics.stdev(list(baseline_results_sub['violations_test_prediction']))
        mean_counter_examples = statistics.mean(list(baseline_results_sub['counter_examples_found']))
        mean_runtime = statistics.mean(list(baseline_results_sub['runtime']))
        std_runtime = statistics.stdev(list(baseline_results_sub['runtime']))

        processed_baseline_results = processed_baseline_results.append([[a, mean_accuracy_test, std_accuracy_test, 
                                                                         mean_accuracy_test_modified, std_accuracy_test_modified,
                                                                         mean_val_violations, std_val_violations,
                                                                         mean_test_violations, std_test_violations,
                                                                         mean_counter_examples,
                                                                         mean_runtime, std_runtime]])
    
    processed_baseline_results.columns = ['alpha', 'average_test_accuracy', 'std_test_accuracy',
                                          'average_test_accuracy_modified', 'std_test_accuracy_modified', 
                                         'mean val violations', 'stddev val violations',
                                         'mean test violations', 'stddev test violations', 'mean_CE',
                                          'mean_runtime', 'std_runtime']
    processed_baseline_results.reset_index(inplace=True, drop=True)
    selected_alpha = processed_baseline_results['alpha'][list(processed_baseline_results['mean val violations']).index(min(list(processed_baseline_results['mean val violations'])))]
    print(processed_baseline_results)
    print(selected_alpha)
    '''calculating the adversity index only for selecetd alpha'''
    baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(selected_alpha)]
    baseline_results_sub.reset_index(inplace=True, drop=True)
    adversity_index_values = [[], [], []]
    
    for i in range(baseline_results_sub.shape[0]):
        print('Running for Fold:', i + 1)
        weights = list(baseline_results_sub.iloc[i, :])[8]
        weights = [[float(v) for v in w] for w in eval(weights)]
        counter_examples = adversity_index(X, weights, K=K)
        adversity_index_values[0].append(counter_examples[0].shape[0]/X.shape[0])
        adversity_index_values[1].append(counter_examples[1].shape[0]/X.shape[0])
        adversity_index_values[2].append(counter_examples[2].shape[0]/X.shape[0])
        
    out_sbr = out_sbr.append([[f, selected_alpha,
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['mean_runtime'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_runtime'].iloc[0],
                           statistics.mean(adversity_index_values[0]),
                           statistics.stdev(adversity_index_values[0]),
                           statistics.mean(adversity_index_values[1]),
                           statistics.stdev(adversity_index_values[1]),
                           statistics.mean(adversity_index_values[2]),
                           statistics.stdev(adversity_index_values[2])
                          ]])
    
out_sbr.columns = ['experiment', 'selected_alpha', 'average_test_accuracy', 'std_test_accuracy', 
                   'average_test_accuracy_modified', 'std_test_accuracy_modified', 'mean_runtime', 'std_runtime', 
                  'average_adversity_index_0.1', 'std_adversity_index_0.1',
                'average_adversity_index_0.01', 'std_adversity_index_0.01', 'average_adversity_index_0.001', 'std_adversity_index_0.001']
print(out_sbr)
print(statistics.mean(out_sbr['average_test_accuracy']))
print(statistics.mean(out_sbr['std_test_accuracy']))

print(statistics.mean(out_sbr['average_test_accuracy_modified']))
print(statistics.mean(out_sbr['std_test_accuracy_modified']))

print(statistics.mean(out_sbr['average_adversity_index_0.001']))
print(statistics.mean(out_sbr['std_adversity_index_0.001']))

print(statistics.mean(out_sbr['average_adversity_index_0.01']))
print(statistics.mean(out_sbr['std_adversity_index_0.01']))

print(statistics.mean(out_sbr['average_adversity_index_0.1']))
print(statistics.mean(out_sbr['std_adversity_index_0.1']))

print(statistics.mean(out_sbr['mean_runtime']))
print(statistics.mean(out_sbr['std_runtime']))

In [ ]:
out_sl = pd.DataFrame()
for f in folders:
    print('Running for Experiment:', f)
    baseline_results = pd.read_csv(str(f) + '/all_baseline_outputs.csv').iloc[:, 1:]
    baseline_results = baseline_results[(baseline_results['margin/alpha'] != 'PP') & (baseline_results['loss_type'] == 'sl')]
    
    alphas = [0, 1, 2, 5, 10, 50, 100]
    processed_baseline_results = pd.DataFrame()

    for a in alphas:
        baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(a)]
        mean_accuracy_test = statistics.mean(list(baseline_results_sub['accuracy_test_corrected']))
        std_accuracy_test = statistics.stdev(list(baseline_results_sub['accuracy_test_corrected'])) 
        mean_accuracy_test_modified = statistics.mean(list(baseline_results_sub['accuracy_test_modified']))
        std_accuracy_test_modified = statistics.stdev(list(baseline_results_sub['accuracy_test_modified'])) 
        mean_val_violations = statistics.mean(list(baseline_results_sub['violations_val_prediction']))
        std_val_violations = statistics.stdev(list(baseline_results_sub['violations_val_prediction']))
        mean_test_violations = statistics.mean(list(baseline_results_sub['violations_test_prediction']))
        std_test_violations = statistics.stdev(list(baseline_results_sub['violations_test_prediction']))
        mean_counter_examples = statistics.mean(list(baseline_results_sub['counter_examples_found']))
        mean_runtime = statistics.mean(list(baseline_results_sub['runtime']))
        std_runtime = statistics.stdev(list(baseline_results_sub['runtime']))

        processed_baseline_results = processed_baseline_results.append([[a, mean_accuracy_test, std_accuracy_test, 
                                                                         mean_accuracy_test_modified, std_accuracy_test_modified,
                                                                         mean_val_violations, std_val_violations,
                                                                         mean_test_violations, std_test_violations,
                                                                         mean_counter_examples,
                                                                         mean_runtime, std_runtime]])
    
    processed_baseline_results.columns = ['alpha', 'average_test_accuracy', 'std_test_accuracy',
                                          'average_test_accuracy_modified', 'std_test_accuracy_modified', 
                                         'mean val violations', 'stddev val violations',
                                         'mean test violations', 'stddev test violations', 'mean_CE',
                                          'mean_runtime', 'std_runtime']
    processed_baseline_results.reset_index(inplace=True, drop=True)
    selected_alpha = processed_baseline_results['alpha'][list(processed_baseline_results['mean val violations']).index(min(list(processed_baseline_results['mean val violations'])))]
    print(processed_baseline_results)
    print(selected_alpha)
    '''calculating the adversity index only for selecetd alpha'''
    baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(selected_alpha)]
    baseline_results_sub.reset_index(inplace=True, drop=True)
    adversity_index_values = [[], [], []]
    
    for i in range(baseline_results_sub.shape[0]):
        print('Running for Fold:', i + 1)
        weights = list(baseline_results_sub.iloc[i, :])[8]
        weights = [[float(v) for v in w] for w in eval(weights)]
        counter_examples = adversity_index(X, weights, K=K)
        adversity_index_values[0].append(counter_examples[0].shape[0]/X.shape[0])
        adversity_index_values[1].append(counter_examples[1].shape[0]/X.shape[0])
        adversity_index_values[2].append(counter_examples[2].shape[0]/X.shape[0])
        
    out_sl = out_sl.append([[f, selected_alpha,
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['mean_runtime'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_runtime'].iloc[0],
                           statistics.mean(adversity_index_values[0]),
                           statistics.stdev(adversity_index_values[0]),
                           statistics.mean(adversity_index_values[1]),
                           statistics.stdev(adversity_index_values[1]),
                           statistics.mean(adversity_index_values[2]),
                           statistics.stdev(adversity_index_values[2])
                          ]])
    
out_sl.columns = ['experiment', 'selected_alpha', 'average_test_accuracy', 'std_test_accuracy', 
                   'average_test_accuracy_modified', 'std_test_accuracy_modified', 'mean_runtime', 'std_runtime', 
                  'average_adversity_index_0.1', 'std_adversity_index_0.1',
                'average_adversity_index_0.01', 'std_adversity_index_0.01', 'average_adversity_index_0.001', 'std_adversity_index_0.001']
print(out_sl)
print(statistics.mean(out_sl['average_test_accuracy']))
print(statistics.mean(out_sl['std_test_accuracy']))

print(statistics.mean(out_sl['average_test_accuracy_modified']))
print(statistics.mean(out_sl['std_test_accuracy_modified']))

print(statistics.mean(out_sl['average_adversity_index_0.001']))
print(statistics.mean(out_sl['std_adversity_index_0.001']))

print(statistics.mean(out_sl['average_adversity_index_0.01']))
print(statistics.mean(out_sl['std_adversity_index_0.01']))

print(statistics.mean(out_sl['average_adversity_index_0.1']))
print(statistics.mean(out_sl['std_adversity_index_0.1']))

print(statistics.mean(out_sl['mean_runtime']))
print(statistics.mean(out_sl['std_runtime']))

In [ ]:
out_pp = pd.DataFrame()
for f in folders:
    print('Running for Experiment:', f)
    baseline_results = pd.read_csv(str(f) + '/all_baseline_outputs.csv').iloc[:, 1:]
    baseline_results = baseline_results[(baseline_results['margin/alpha'] == 'PP') & (baseline_results['loss_type'] == 'sl')]
    
    alphas = ['PP']
    processed_baseline_results = pd.DataFrame()

    for a in alphas:
        baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(a)]
        mean_accuracy_test = statistics.mean(list(baseline_results_sub['accuracy_test_corrected']))
        std_accuracy_test = statistics.stdev(list(baseline_results_sub['accuracy_test_corrected'])) 
        mean_accuracy_test_modified = statistics.mean(list(baseline_results_sub['accuracy_test_modified']))
        std_accuracy_test_modified = statistics.stdev(list(baseline_results_sub['accuracy_test_modified'])) 
        mean_val_violations = statistics.mean(list(baseline_results_sub['violations_val_prediction']))
        std_val_violations = statistics.stdev(list(baseline_results_sub['violations_val_prediction']))
        mean_test_violations = statistics.mean(list(baseline_results_sub['violations_test_prediction']))
        std_test_violations = statistics.stdev(list(baseline_results_sub['violations_test_prediction']))
        mean_counter_examples = statistics.mean(list(baseline_results_sub['counter_examples_found']))
        mean_runtime = statistics.mean(list(baseline_results_sub['runtime']))
        std_runtime = statistics.stdev(list(baseline_results_sub['runtime']))

        processed_baseline_results = processed_baseline_results.append([[a, mean_accuracy_test, std_accuracy_test, 
                                                                         mean_accuracy_test_modified, std_accuracy_test_modified,
                                                                         mean_val_violations, std_val_violations,
                                                                         mean_test_violations, std_test_violations,
                                                                         mean_counter_examples,
                                                                         mean_runtime, std_runtime]])
    
    processed_baseline_results.columns = ['alpha', 'average_test_accuracy', 'std_test_accuracy',
                                          'average_test_accuracy_modified', 'std_test_accuracy_modified', 
                                         'mean val violations', 'stddev val violations',
                                         'mean test violations', 'stddev test violations', 'mean_CE',
                                          'mean_runtime', 'std_runtime']
    processed_baseline_results.reset_index(inplace=True, drop=True)
    selected_alpha = processed_baseline_results['alpha'][list(processed_baseline_results['mean val violations']).index(min(list(processed_baseline_results['mean val violations'])))]
    print(processed_baseline_results)
    print(selected_alpha)
    '''calculating the adversity index only for selecetd alpha'''
    baseline_results_sub = baseline_results[baseline_results['margin/alpha'] == str(selected_alpha)]
    baseline_results_sub.reset_index(inplace=True, drop=True)
    adversity_index_values = [[], [], []]
    
    for i in range(baseline_results_sub.shape[0]):
        print('Running for Fold:', i + 1)
        weights = list(baseline_results_sub.iloc[i, :])[8]
        weights = [[float(v) for v in w] for w in eval(weights)]
        counter_examples = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
        adversity_index_values[0].append(counter_examples[0].shape[0]/X.shape[0])
        adversity_index_values[1].append(counter_examples[1].shape[0]/X.shape[0])
        adversity_index_values[2].append(counter_examples[2].shape[0]/X.shape[0])
        
    out_pp = out_pp.append([[f, selected_alpha,
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['average_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_test_accuracy_modified'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['mean_runtime'].iloc[0],
                           processed_baseline_results[processed_baseline_results['alpha'] == selected_alpha]['std_runtime'].iloc[0],
                           statistics.mean(adversity_index_values[0]),
                           statistics.stdev(adversity_index_values[0]),
                           statistics.mean(adversity_index_values[1]),
                           statistics.stdev(adversity_index_values[1]),
                           statistics.mean(adversity_index_values[2]),
                           statistics.stdev(adversity_index_values[2])
                          ]])
    
out_pp.columns = ['experiment', 'selected_alpha', 'average_test_accuracy', 'std_test_accuracy', 
                   'average_test_accuracy_modified', 'std_test_accuracy_modified', 'mean_runtime', 'std_runtime', 
                  'average_adversity_index_0.1', 'std_adversity_index_0.1',
                'average_adversity_index_0.01', 'std_adversity_index_0.01', 'average_adversity_index_0.001', 'std_adversity_index_0.001']
print(out_pp)
print(statistics.mean(out_pp['average_test_accuracy']))
print(statistics.mean(out_pp['std_test_accuracy']))

print(statistics.mean(out_pp['average_test_accuracy_modified']))
print(statistics.mean(out_pp['std_test_accuracy_modified']))

print(statistics.mean(out_pp['average_adversity_index_0.001']))
print(statistics.mean(out_pp['std_adversity_index_0.001']))

print(statistics.mean(out_pp['average_adversity_index_0.01']))
print(statistics.mean(out_pp['std_adversity_index_0.01']))

print(statistics.mean(out_pp['average_adversity_index_0.1']))
print(statistics.mean(out_pp['std_adversity_index_0.1']))

print(statistics.mean(out_pp['mean_runtime']))
print(statistics.mean(out_pp['std_runtime']))

In [ ]:
out_l = pd.DataFrame()
for f in folders:
    results = pd.read_csv(str(f) + '/all_outputs_sade_l.csv').iloc[:, 1:]
    print(results)
    mean_runtime = statistics.mean([float(r) for r in results['runtime']])
    std_runtime = statistics.stdev([float(r) for r in results['runtime']])
    out_l = out_l.append([[f, statistics.mean(list(results['accuracy_test_corrected'])), 
                           statistics.stdev(list(results['accuracy_test_corrected'])),
                           statistics.mean(list(results['accuracy_test_modified'])), 
                           statistics.stdev(list(results['accuracy_test_modified'])),
                           mean_runtime, std_runtime]])
out_l.columns = ['experiment', 'average_test_accuracy', 'std_test_accuracy', 
                 'average_test_accuracy_modified', 'std_test_accuracy_modified',
                 'mean_runtime', 'std_runtime']
print(out_l)
print(statistics.mean(out_l['average_test_accuracy']))
print(statistics.mean(out_l['std_test_accuracy']))
print(statistics.mean(out_l['average_test_accuracy_modified']))
print(statistics.mean(out_l['std_test_accuracy_modified']))

print(statistics.mean(out_l['mean_runtime']))
print(statistics.mean(out_l['std_runtime']))

### Plots

In [ ]:
import pickle
import matplotlib.pyplot as plt

i = 0
for f in folders:
    i = i+1
    with (open(str(f) + "/all_test_losses_sade_l.pkl", "rb")) as openfile:
        out = pickle.load(openfile)
    out = [[float(i) for i in l] for l in out]

    folds = 5

    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    plt.rcParams["axes.titleweight"] = "bold"
    plt.rcParams['font.size'] = 14
    plt.rc('figure', figsize=(10, 5))

    for e in range(folds):
        plt.figure()
        plt.plot(out[e])
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.title('Music Genre Classification (experiment = {}, fold = {})'.format(folders[i-1], e + 1))
        name = str(f) + '/music_data_{}.jpeg'.format(e+1)
        plt.savefig('{}'.format(name))
        plt.show()
        plt.close()